# Georeferencing

### This code was modified from Micasense: https://github.com/micasense/imageprocessing
To use it, you have to set the Micasense conda environment prior to the processing steps.

In [ ]:
import cameratransform as ct

for n in range(1):
    log = pd.read_csv(os.path.join(imagePath, 'flight_stacks', 'log.csv'), index_col=False) # The log.csv contains geoinformation of the captures

    focal = log['FocalLength'][n]
    image_size = log['ImageWidth'][n], log['ImageHeight'][n]
    lat = log['GPSLatitude'][n]
    lon = log['GPSLongitude'][n]
    alt = log['GPSAltitude'][n]
    pitch = log['Pitch'][n]
    roll = log['Roll'][n]
    yaw = log['Yaw'][n]

    cam = ct.Camera(ct.RectilinearProjection(focallength_mm=focal,
                                         image=image_size),
                   ct.SpatialOrientation(elevation_m=alt,
                                     tilt_deg=0,
                                     roll_deg=0,
                                    heading_deg=270)) # This parameter should be set as 0º (when the UAV is heading East), 90º (when the UAV is heading North), 180º (when the UAV is heading West), or 270º (when the UAV is heading South). 

    # Latitude and Longitude of the GPS
    cam.setGPSpos(lat, lon, alt)
    
    # Image corners coordinates
    coords = np.array([cam.gpsFromImage([0 , 0]), \
        cam.gpsFromImage([image_size[0]-1 , 0]), \
        cam.gpsFromImage([image_size[0]-1, image_size[1]-1]), \
        cam.gpsFromImage([0 , image_size[1]-1])])

    gcp1 = rasterio.control.GroundControlPoint(row=0, col=0, x=coords[0,1], y=coords[0,0], z=coords[0,2], id=None, info=None)
    gcp2 = rasterio.control.GroundControlPoint(row=image_size[0]-1, col=0, x=coords[1,1], y=coords[1,0], z=coords[1,2], id=None, info=None)
    gcp3 = rasterio.control.GroundControlPoint(row=image_size[0]-1, col=image_size[1]-1, x=coords[2,1], y=coords[2,0], z=coords[2,2], id=None, info=None)
    gcp4 = rasterio.control.GroundControlPoint(row=0, col=image_size[1]-1, x=coords[3,1], y=coords[3,0], z=coords[3,2], id=None, info=None)

    # Opening the original Image and generating a profile based on flight_stacks file generated before
    with rasterio.open(os.path.join(imagePath, 'flight_stacks', log['ID'][n]), 'r') as src:
        profile = src.profile

        # Transformation
        tsfm = rasterio.transform.from_gcps([gcp1,gcp2,gcp3,gcp4])
    
        crs = rasterio.crs.CRS({"init": "epsg:4326"})

        profile.update(dtype=rasterio.uint8, transform = tsfm, crs=crs)

        with rasterio.open(os.path.join(imagePath, 'georeferenced_stacks', log['ID'][n]), 'w', **profile) as dst:
            print(src.read().shape)
            dst.write(src.read().astype(rasterio.uint8)) # We write the coordinates in the image with this line.